[slides](https://docs.google.com/presentation/d/19K9nVjuSOCrZGM6lmFeEEarTm2xZwDSiZEIzf-Ywr5o/edit?usp=sharing)

[python-ecdsa docs](https://github.com/warner/python-ecdsa)

# Signing our First Message with ECDSA

In [14]:
from ecdsa import SigningKey, SECP256k1
private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key()

m = b'I am hungry'
signature = private_key.sign(m)
print(signature)

b'r\x174(\x03Z\xa1\r5\xe3\x01\x1e\x88\x98o\rJ\xa2\x1a\xfb\x82?\xaa\xeb\xa21\xcf\x9d=\x13(\xd8S\xe8x\xca\xed\xc5\x06>0x\xd1P\xa9\xc4\xbcQ\xd9\x05j$L\x13\xc2dN\xc2eR\xee\xdaYM'


In [16]:
public_key.verify(signature, m)

True

In [18]:
public_key.verify(signature, b'Getting tired')

BadSignatureError: 

# Defining ECDSACoin

* A coin is just a list of transfers, just like with PNGCoin. 
    * Where transfers were photographs of signatures in PNGCoin, they are ECDSA digital signatures in ECDSACoin
* The `public_key` in the last transfer is who owns the coin
* To spend the coin append a new transfer. Use the public key of the person you are sending to, and sign it using your private key.

In [37]:
class Transfer:
    
    def __init__(self, signature, public_key):
        self.signature = signature
        self.public_key = public_key
        
class ECDSACoin:
    def __init__(self, transfers):
        self.transfers = transfers

In [55]:
# The usual suspects ... 
# SECP256k1 is a detail about the "magical multiplication" used under the covers

bank_private_key = SigningKey.generate(curve=SECP256k1)
bob_private_key = SigningKey.generate(curve=SECP256k1)
alice_private_key = SigningKey.generate(curve=SECP256k1)

bank_public_key = bank_private_key.get_verifying_key()
bob_public_key = bob_private_key.get_verifying_key()
alice_public_key = alice_private_key.get_verifying_key()

In [131]:
from utils import serialize
def issue(public_key):
    
    message = serialize(public_key)
    signature = bank_private_key.sign(message)
    
    transfer = Transfer(
        signature = signature,
        public_key = public_key,
    )
    coin = ECDSACoin(transfers=[transfer])
    return coin

# Validating the First Transfer

In [35]:
def validate(coin):
    trans = coin.transfers[0]
    
    message = serialize(trans.public_key)
    bank_public_key.verify(trans.signature, message)

In [138]:
# create a coin
alice_coin = issue(alice_public_key)

# validate
validate(alice_coin)

coin is valid


In [39]:
# negative test
# create bad coin 
message = serialize(bob_public_key)
signature = bob_private_key.sign(message)
    
transfer = Transfer(
    signature = signature,
    public_key = bob_public_key,
)
alt_coin = ECDSACoin([transfer])

In [139]:
from ecdsa import BadSignatureError

try:
    validate(alt_coin)
except BadSignatureError:
    print('Bad signature')
    

Bad signature


# Validating Subsequent Transfers

In [144]:
def transfer_message(pre_sig, next_pub):
    return serialize({
        "previous_signature": pre_sig,
        "next_owner_public_key": next_pub,
    })

def validate_full(coin):
    # check first transfer (special, message is just public key)
    trans = coin.transfers[0]
    message = serialize(trans.public_key)
    bank_public_key.verify(trans.signature, message)
    
    # check the rest of the transfers
    previous_transfer = coin.transfers[0]
    for next_transfer in coin.transfers[1:]:
        
        message = transfer_message(previous_transfer.signature, next_transfer.public_key)
        previous_transfer.public_key.verify(
            next_transfer.signature,
            message
        )
        previous_transfer = next_transfer
    print('coin is valid')

def get_owner(coin):
    database = {
        serialize(bob_public_key): "Bob", 
        serialize(alice_public_key): "Alice",
        serialize(bank_public_key): "Bank"
    }
    public_key = serialize(coin.transfers[-1].public_key)
    return database[public_key]

In [146]:
coin = issue(alice_public_key)
validate_full(coin)

print('This coin is owned by', get_owner(coin))
message = transfer_message(coin.transfers[-1].signature, bob_public_key)
signature = alice_private_key.sign(message)

a_b_transfer = Transfer(
    signature,
    bob_public_key
)
coin.transfers.append(a_b_transfer)
validate_full(coin)
print('This coin is owned by', get_owner(coin))

message = transfer_message(coin.transfers[-1].signature, alice_public_key)
signature = bob_private_key.sign(message)

b_b_transfer = Transfer(
    signature,
    alice_public_key
)
coin.transfers.append(b_b_transfer)
validate_full(coin)
print('This coin is owned by', get_owner(coin))


coin is valid
This coin is owned by Alice
coin is valid
This coin is owned by Bob
coin is valid
This coin is owned by Alice


# Serialization

In [113]:
from utils import to_disk, from_disk
import os

In [114]:
filename = "coin.ecdsa"

print("does the coinfile exists?", os.path.isfile(filename))

coin = issue(alice_public_key)

to_disk(coin, filename)

does the coinfile exists? False


In [123]:
print("does the coinfile exists?", os.path.isfile(filename))

does the coinfile exists? True


In [122]:
coin2 = from_disk(filename)
coin
coin2

# The Finished Product

[ecdsacoin.py](ecdsacoin.py)

In [147]:
import ecdsacoin

coin = ecdsacoin.issue(alice_public_key)
coin.validate()

alice_to_bob = Transfer(
    signature=alice_private_key.sign(transfer_message(coin.transfers[-1].signature, bob_public_key)),
    public_key=bob_public_key,
)

coin.transfers.append(alice_to_bob)
coin.validate()